# Anexo I: Transfromación y Extracción de Temperatura

## Instalación de Paquetes Requeridos

Se trabaja con la API de Google Earth Engine para el acceso y procesamiento de imágenes satelitales.

In [ ]:
!pip install google-api-python-client
!pip install earthengine-api
!earthengine authenticate

## Importar Paquetes

Se importan las librerías necesarias y se inicializa la API de Google Earth Engine.

In [ ]:
from datetime import datetime as dt
import pandas as pd
import numpy as np
import ee

ee.Initialize()

## Cálculo de Temperatura Media por Imagen

Se define una función que corta las imágenes a la extensión de Colombia, selecciona las imágenes de temperatura máxima y mínima, calcula la media como la suma de ambas dividida en 2, la multiplica por 0.02 (factor de conversión para obtener la temperatura en Kelvin) y se divide por 273.15 para tenerla en Celsius.

In [ ]:
def setMeanTemperature(image):  
  img = image.clip(COL) # CORTAR A LA EXTENSION DE COLOMBIA
  TMIN = img.select('LST_Night_1km') # SELECCIONAR BANDA DE TEMPERATURA MINIMA
  TMAX = img.select('LST_Day_1km') # SELECCIONAR BANDA DE TEMPERATURA MAXIMA
  TMED = TMIN.add(TMAX).divide(2).multiply(0.02).subtract(273.15) # CALCULAR Y TRANSFORMAR TEMPERATURA A MEDIA CELSIUS
  return TMED.copyProperties(img,['system:time_start']) # RETORNAR IMAGEN MANTENIENDO LAS PROPIEDADES ORIGINALES

## Definición de Variables Necesarias

Colección espacial de la extensión de Colombia, colección espacial de la cobertura de cada municipio de Colombia, nombres de cada municipio y la cantidad de departamentos.

In [ ]:
COL = ee.FeatureCollection('users/dmlmont/Colombia') # COLECCION ESPACIAL DE COLOMBIA
MUN = ee.FeatureCollection('users/dmlmont/Municipios_2017') # COLECCION ESPACIAL DE MUNICIPIOS
NAMES = ee.List(MUN.aggregate_array("NOM_DEP")).distinct(); # NOMBRES DE CADA MUNICIPIO
TOTAL_DEP = NAMES.size().getInfo() # CANTIDAD DE DEPARTAMENTOS

## Agregación de Temperatura por Semestre

Se obtiene la colección de imágenes de temperatura de MODIS y se calcula la temperatura media por semestre.

In [ ]:
MODIS = ee.ImageCollection('MODIS/006/MOD11A1') # TODA LA COLECCION DE TEMPERATURA DE MODIS
  .filterDate('2015-01-01','2018-06-30') # SE FILTRA POR LA COBERTURA TEMPORAL NECESARIA
  .filterBounds(COL) # SE FILTRA POR LA COBERTURA ESPACIAL NECESARIA
  .map(setMeanTemperature) # SE UTILIZA LA FUNCION CREADA PARA OBTENER TEMPERATURA MEDIA POR IMAGEN

# TEMPERATURA 2015
MOD2015A = MODIS.filterDate('2015-01-01','2015-06-30').mean().set('system:time_start',ee.Date('2015-01-01'));
MOD2015B = MODIS.filterDate('2015-07-01','2015-12-31').mean().set('system:time_start',ee.Date('2015-07-01'));

# TEMPERATURA 2016
MOD2016A = MODIS.filterDate('2016-01-01','2016-06-30').mean().set('system:time_start',ee.Date('2016-01-01'));
MOD2016B = MODIS.filterDate('2016-07-01','2016-12-31').mean().set('system:time_start',ee.Date('2016-07-01'));

# TEMPERATURA 2017
MOD2017A = MODIS.filterDate('2017-01-01','2017-06-30').mean().set('system:time_start',ee.Date('2017-01-01'));
MOD2017B = MODIS.filterDate('2017-07-01','2017-12-31').mean().set('system:time_start',ee.Date('2017-07-01'));

# TEMPERATURA 2018
MOD2018A = MODIS.filterDate('2018-01-01','2018-06-30').mean().set('system:time_start',ee.Date('2018-01-01'));

# LISTA DE IMAGENES OBTENIDAS (7 IMAGENES EN TOTAL)
# Y NOMBRES PARA AGREGAR COMO COLUMNAS A LA BASE DE DATOS
MOD_IMAGES = [MOD2015A,MOD2015B,MOD2016A,MOD2016B,MOD2017A,MOD2017B,MOD2018A]
MOD_IMAGES_NAMES = ["MOD2015A","MOD2015B","MOD2016A","MOD2016B","MOD2017A","MOD2017B","MOD2018A"]

## Extracción de Temperatura por Municipio

Se extrae el promedio de temperatura de los pixeles que están dentro de cada municipio.

In [ ]:
df = pd.DataFrame() # DATA FRAME VACIO PARA GUARDAR DATOS

# SE HARA UN CICLO PARA CADA DEPARTAMENTO
# SI SE PASAN TODOS LOS MUNICIPIOS GOOGLE ARROJA LIMITE DE MEMORIA EXCEDIDO
for dept in range(TOTAL_DEP):

  M = MUN.filter(ee.Filter.eq('NOM_DEP',NAMES.get(dept))) # MUNICIPIOS DEL DEPARTAMENTO
  features = M.getInfo()['features'] # FEATURES DEL DEPARTAMENTO

  data = list() # LISTA VACIA PARA GUARDAR DATOS POR DEPATRAMENTP

  # RECORREMOS TODAS LAS FEATURES DEL DEPARTAMENTO (MUNICIPIOS)
  for f in range(len(features)):
    
    geom = features[f]['geometry'] # GEOMETRIA ESPACIAL DE CADA MUNICIPIO

    # CHEQUEAMOS QUE LAS GEOMETRIAS SEAN POLIGONOS O POLIGONOS DENTRO DE UNA COLECCION DE GEOMETRIAS
    # SI UN MUNICIPIO TIENE GEOMETRIAS QUE NO SON POLIGONOS LAS OBVIAMOS
    if geom['type'] == 'Polygon':
      polygon = ee.Geometry.Polygon(geom['coordinates'])
    elif geom['type'] == 'GeometryCollection':
      for g in range(len(geom['geometries'])):
        if geom['geometries'][g]['type'] == 'Polygon':
          polygon = ee.Geometry.Polygon(geom['geometries'][g]['coordinates'])
        else:
          pass
    else:
      pass
    
    featureData = features[f]['properties'] # EXTRAEMOS LOS DATOS ASOCIADOS A CADA MUNICIPIO (NOMBRE, DEPARTAMENTO, ETC.)
    fc = ee.FeatureCollection(polygon) # CONVERTIMOS EL POLIGONO A UNA COLECCION ESPACIAL

    # PARA CADA IMAGEN VAMOS A CALCULAR EL PROMEDIO DE PIXELES POR MUNICIPIO
    for i in range(len(MOD_IMAGES)):
      img = MOD_IMAGES[i]  # IMAGEN DE TEMPERATURA i
      imgName = MOD_IMAGES_NAMES[i] # NOMBRE DE LA IMAGEN i
      tmean_dictionary = img.reduceRegion(ee.Reducer.mean(),fc,1000) # EXTRAER PROMEDIO DE TEMPERATURA EN UN DICCIONARIO
      tmean_value = tmean_dictionary.get('LST_Night_1km') # EXTRAER EL VALOR DEL DICCIONARIO
      featureData[imgName] = tmean_value.getInfo() # AGREGAR UNA NUEVA COLUMNA A LOS DATOS DE CADA MUNICIPIO CON LA TEMPERATURA

    keys = list(featureData.keys()) # GUARDAR LAS KEYS PARA USARLAS COMO NOMBRES
    values = list(featureData.values()) # GUARDAR LOS VALORES DE CADA DATO
    data.append(values) # AGREGAR A LA LISTA VACIA

  df = df.append(pd.DataFrame(data,columns = keys),ignore_index = True) # CREAR BASE DE DATOS COMPLETA

## Exportar Datos

In [ ]:
from google.colab import drive # IMPORTAR DRIVE PARA EXPORTAR DATOS (SOLO EN LINEA, EN LOCAL NO ES NECESARIO)

drive.mount('/content/drive') # AUTORIZAR DRIVE

In [ ]:
df.to_csv(r'/content/drive/My Drive/DLC.csv',index = False) # EXPORTAR DATOS A UN CSV